In [39]:
import xlrd
import numpy as np

#read class
def readTranscript(filename='genes_transcript.xlsx'):
    sheet = xlrd.open_workbook(filename).sheet_by_index(0)
    nRows = sheet.nrows
    my_class_label = []
    my_class_value = []
    for rowNum in range(0,nRows):
        rowList = []
        row = sheet.row_values(rowNum)
        rowLabel = row.pop(0)
        my_class_label.append([rowLabel])
        for el in row:
            rowList.append([el])
        rowList = np.array(rowList)
        my_class_value.append(rowList)
    my_class_label = np.array(my_class_label)
    my_class_value = np.array(my_class_value)
    #print(my_class_label)
    #print(my_class_value)
    return my_class_label, my_class_value

#read data
def readData(filename='gene_data.xlsx'):
    sheet = xlrd.open_workbook(filename).sheet_by_index(0)
    nRows = sheet.nrows
    my_data_value = []
    for rowNum in range(0,nRows):
        rowList = []
        row = sheet.row_values(rowNum)
        for el in row:
            rowList.append([el])
        rowList = np.array(rowList)
        my_data_value.append(rowList)
    my_data_value = np.array(my_data_value)
    #print(my_data_value)
    return my_data_value

class_name, class_value = readTranscript()
data_value = readData()

In [87]:
#Chi square feature selection

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

#select 100 features
def chisq_feature_selection(X, y,numoffeature=100):
    #normalize data
    #mms = MinMaxScaler()
    #Xnorm = mms.fit_transform(X)
    Xnorm = X
    #chi square
    chisq = SelectKBest(chi2, k=numoffeature)
    Xchi = chisq.fit_transform(Xnorm, y)
    return Xchi


In [41]:
data = np.copy(data_value)
#print(data)

In [43]:
#remove header(name of each sample)
data1 = np.delete(data, 0,axis=0)
#print(data1)

#remove first row(ID row)
data2 = np.delete(data1, 0, axis=1)
#print(data2)

[[['0.0']
  ['0.0271214']
  ['0.0']
  ...
  ['0.0']
  ['0.0']
  ['0.0']]

 [['0.011070568']
  ['0.00752636']
  ['0.0']
  ...
  ['0.0']
  ['0.0']
  ['0.0']]

 [['3.9065173']
  ['2.6216204']
  ['2.5022274']
  ...
  ['2.6509483']
  ['3.1214452']
  ['1.6998575']]

 ...

 [['0.0']
  ['0.0']
  ['0.0']
  ...
  ['0.0']
  ['0.0']
  ['0.0']]

 [['0.37584436']
  ['0.43296269']
  ['0.2109373']
  ...
  ['0.28897299']
  ['0.13009639']
  ['0.25719846']]

 [['0.0']
  ['0.0']
  ['0.0']
  ...
  ['0.0']
  ['0.0']
  ['0.0']]]


In [74]:
#select gleason (total) score
y = class_value[3]
#print(y)

[['9.0']
 ['6.0']
 ['8.0']
 ['7.0']
 ['9.0']
 ['7.0']
 ['6.0']
 ['6.0']
 ['8.0']
 ['7.0']
 ['6.0']
 ['7.0']
 ['7.0']
 ['9.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['10.0']
 ['8.0']
 ['9.0']
 ['9.0']
 ['9.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['6.0']
 ['9.0']
 ['7.0']
 ['9.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['9.0']
 ['7.0']
 ['7.0']
 ['8.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['8.0']
 ['7.0']
 ['7.0']
 ['8.0']
 ['9.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['9.0']
 ['7.0']
 ['7.0']
 ['6.0']
 ['9.0']
 ['8.0']
 ['7.0']
 ['7.0']
 ['9.0']
 ['9.0']
 ['8.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['6.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['8.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['7.0']
 ['8.0']
 ['9.0']
 ['7.0']
 ['8.0']
 ['8.0']
 ['8.0']
 ['8.0']
 ['8.0']


In [93]:
#turn 3dd list to 2d list
#print(data2.shape)
#data3 = np.reshape(data2, (data2.shape[0], data2.shape[1],))
#print(data3.shape)

#transpose to match the class
data4 = data3.transpose()
#print(data4)

#change from str to float
data_float = data4.astype(np.float)
#print(data_float)

#change from str to float
y_float = y.astype(np.float)
#print(y_float)

X = chisq_feature_selection(data_float, y_float)
print(X.shape)

(494, 100)


In [ ]:

from sklearn.model_selection import train_test_split